In [1]:
import requests
import os
url = "https://huggingface.co/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
r = requests.get(url)
os.makedirs("./cbdb_sqlite", exist_ok=True)
with open("./cbdb_sqlite/latest.7z", "wb") as f:
    f.write(r.content)

Cloning into 'cbdb_sqlite'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 31 (delta 3), reused 9 (delta 3), pack-reused 22
Unpacking objects: 100% (31/31), done.


In [2]:
# import glob
import os
latest_file = "./cbdb_sqlite/latest.7z"
latest_db = latest_file.split("/")[2].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

latest.db


In [3]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(latest_file).extractall(".")

     |████████████████████████████████| 77 kB 5.3 MB/s 


In [4]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [5]:
entry_df = pd.read_sql_query("SELECT c_entry_desc_chn FROM ENTRY_CODES", CONN).drop_duplicates().dropna()

In [6]:
entry_df

,c_entry_desc_chn
0,[Missing Data]
1,未知
2,前帝遜位
3,臨時保留，待考。
4,胥吏出職
...,...
263,介賓
264,宗室選試: 量試
265,宗室選試: 取應
266,賜名;賜名授官


In [7]:
def names_massage(df, column):
  import re
  # filter out meaningless entries
  skip_words = ['Missing Data', '未知', '刪除', '馬甲', '其他', '布衣', '民人', '書院', '俊秀', '披甲', '外戚', '宗室', '覺羅', '得度', '閑散', '前鋒', '護軍', '行伍', '領催']
  skip_words_as_regex = '|'.join(skip_words)
  skip_phrases_as_regex = '臨時保留，待考。|刪除|以文學、文藝補官|以茂才\,才行\,武功\,廉仁等科舉\/授官 （非制科）|吏部試 (隋唐)|科舉\: 三史|科舉\: 九經|科舉\: 五經|科舉制舉\: 百篇|內務府 （清朝）|宗教\: 受洗|宗教\: 做喇嘛|於國內公立私立大學就學、畢業|封土建藩\(始封\)|皇子\/親王|科舉\: 詞科|清朝\: 考取官位|吏部試 \(隋唐\)'
  all_skip_as_regex = '|'.join([skip_words_as_regex, skip_phrases_as_regex])
  df = df[df[column].str.contains(all_skip_as_regex)==False]
  # filter other weirdos
  df = df[df[column].str.contains('.*[a-zA-Z]+.*')==False]
  # special replacements
  #  1-1 replacement
  replace_dict = {
      '科舉制舉: 秀才(宋及以前)': '秀才',
      '特賜補官、准赦文除授等': '特賜補官、准赦文除授',
      '科舉學校: 恩賜出身、特賜出身等': '科舉學校: 恩賜出身、特賜出身',
      '科舉: 正奏名諸科(不含明經、明法)': '正奏名諸科',
      '貢生 = 貢監生(籠統) (明清賓貢,功貢)': '貢生、貢監生、賓貢、功貢',
      '貢生: 選貢(優貢 after 1628)': '貢生、選貢、優貢',
      '監生: 蔭監(含明代官生、恩生，或清代恩廕生、難廕生)': '監生、蔭監、官生、恩生、恩廕生、難廕生',
      '其他蔭補(確定非聖節、大禮、致仕、遺表、死事蔭補)': '蔭補'
  }
  for key, val in replace_dict.items():
    df[column] = df[column].replace(key, val)
  #  regex replacement
  df[column] = df[column].str.replace("\(籠統\)", "", regex=True)
  # break line
  df[column] = df[column].apply(lambda x: re.split('、|\(|\)|（|）|：|\:|\/|\=|\:| |\,|；|;', x))
  df = df.explode(column).drop_duplicates().dropna()
  # filter out meaningless entries again
  df = df[df[column].str.contains('學校')==False]
  # replace spaces with blank
  df[column] = df[column].str.replace("\\s", "", regex=True)
  # remove single-character name
  df = df[df[column].apply(lambda x: len(x)>1)]
  # remove specific entries 
  skip_exact_words = set(['進士'])
  df = df[df[column].apply(lambda x: x not in skip_exact_words)]
  df.drop_duplicates().dropna()
  return df

In [8]:
filtered_entry_df = names_massage(entry_df, 'c_entry_desc_chn')

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [9]:
filtered_entry_df

,c_entry_desc_chn
2,前帝遜位
4,胥吏出職
5,進納補官
5,捐授
6,恩蔭
...,...
262,大賓
263,介賓
266,賜名
266,賜名授官


In [10]:
# Sort by descending order of the length
final_df = filtered_entry_df.sort_values(by="c_entry_desc_chn", key=lambda x: -x.str.len())
final_df

,c_entry_desc_chn
219,清廉守節政術可稱堪縣令科
177,賢良方正能直言極諫科
228,詳明政術可以理人科
201,才高未達沈跡下僚科
235,高才沈淪草澤自舉科
...,...
88,例監
161,射策
84,副貢
18,制舉


In [11]:
final_df.to_csv('cbdb_entity_entries.csv', index=False)

In [12]:
# close the db
CONN.close()